In [34]:

import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import numpy as np

In [35]:
perfumes = pd.read_csv(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\perfumes_batch1.csv")
perfumes

,Absolute URLs,Name,Brand,Ingredients,Classification,Family_and_Subfamily,Description
0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed..."
1,https://www.wikiparfum.com/en/fragrances/o-fem...,O FEMININ,M. Micallef,"Rose, Neroli, Musk, Pink Pepper, Patchouli, Le...",Floral Crisp Citrus,"FLORAL, CITRUS","Sensual, carnal and modern, this chic and bohe..."
2,https://www.wikiparfum.com/en/fragrances/miss-...,MISS DIOR PARFUM,Dior,"Cedarwood (Alaska), Patchouli, Mandarin, Straw...",Floral Crisp Fruity,"FLORAL, FRUITY","Between strength and grace, between recklessne..."
3,https://www.wikiparfum.com/en/fragrances/la-vi...,LA VIE EST BELLE ROSE EXTRAORDINAIRE,Lancome,"Rose, Musk, Woody Notes, Iris / Orris, Rose Da...",Floral Classical Rose,FLORAL,La Vie Est Belle Rose Extraordinaire is a rein...
4,https://www.wikiparfum.com/en/fragrances/karl-...,KARL LAGERFELD POUR FEMME ROUGE,Karl Lagerfeld,"Pear, Patchouli, Peach, Vanilla, Blackcurrant,...",Floral Amber Crisp Fruity,"AMBERY (ORIENTAL), FLORAL","Rouge is ""unique, modern and captivating,"" as ..."
...,...,...,...,...,...,...,...
10822,https://www.wikiparfum.com/en/fragrances/genet...,GENETIC BLISS,27 87,"Belambre (Woody ambery), Akigalawood (Patchoul...",Woods Classical,WOODY,"Genetic Bliss belongs to the collection ""Next ..."
10823,https://www.wikiparfum.com/en/fragrances/hamaca,HAMACA,27 87,"Coconut, Tonka Bean, Vanilla, Sandalwood, Marine",Amber Fresh Fruity,"AMBERY (ORIENTAL), FRUITY","A gentle breeze on salty skin, the soft warmth..."
10824,https://www.wikiparfum.com/en/fragrances/wande...,WANDERVOGEL,27 87,"Musk, Mint, Fennel Seeds, Shiso Leaves, Floral...",Soft Floral Fresh Green,"FLORAL, GREEN","It is the irresistible urge to get on a plane,..."
10825,https://www.wikiparfum.com/en/fragrances/elixi...,ELIXIR DE BOMBE,27 87,"Tangerine, Ginger, Hot Pepper, Raspberry, Ylan...",Dry Woods Fresh Fruity,"FRUITY, LEATHER","Boom, there it is, the right moment when you g..."


In [36]:
cols = []
for i in range(len(perfumes.columns)): 
    cols.append(perfumes.columns[i].lower().replace(' ', '_')) 
perfumes.columns = cols

In [37]:

perfumes.duplicated().sum()


0

In [38]:
perfumes.isna().sum()


absolute_urls            0
name                     8
brand                    8
ingredients              8
classification          10
family_and_subfamily     8
description             16
dtype: int64

In [39]:
perfumes = perfumes.dropna(subset=['name'])

In [40]:
perfumes.isna().sum()


absolute_urls           0
name                    0
brand                   0
ingredients             0
classification          2
family_and_subfamily    0
description             8
dtype: int64

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
def custom_tokenizer(text):
    return text.split(',')
count_vectorizer = CountVectorizer(tokenizer=custom_tokenizer)
bag_of_words = count_vectorizer.fit_transform(perfumes.ingredients)
bag_of_words.shape




(10819, 1457)

In [42]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
import pickle, os, sys

similarity_matrix_sparse = cosine_similarity(bag_of_words, dense_output=False)
similarity_matrix_sparse.shape


(10819, 10819)

In [43]:
max_values = 5
num_rows = similarity_matrix_sparse.shape[0]

vect_index = []
vect_values = []

for index in range(num_rows):
    if index%5000 == 0 and index !=0: print(f'{index} calculated perfumes')
    if index+1 == num_rows: print(f'{index} calculated perfumes')
    perfume_search = similarity_matrix_sparse.getrow(index)
    top_similarity = np.argsort(perfume_search.data)[-max_values:][::-1]
    vect_index.append(perfume_search.indices[top_similarity])
    vect_values.append(perfume_search.data[top_similarity])

5000 calculated perfumes
10000 calculated perfumes
10818 calculated perfumes


In [44]:
# Assuming similarity_matrix_sparse is your sparse similarity matrix
# and perfumes is a DataFrame containing your product data, with columns 'name' and 'brand'

def find_similar_products(similarity_matrix_sparse, perfumes, query_product_id, max_values=5):
    # Get the similarity row corresponding to the query product
    query_product_similarity = similarity_matrix_sparse.getrow(query_product_id)
    
    # Get the top similar products indices
    top_similar_products = np.argsort(query_product_similarity.data)[-max_values-1:-1][::-1]  # Exclude the query product itself
    
    # Get the indices of similar products
    similar_product_ids = query_product_similarity.indices[top_similar_products]
    
    # Get the combined name and brand of similar products
    similar_product_names = [perfumes.loc[id, 'name'] + ' ' + perfumes.loc[id, 'brand'] for id in similar_product_ids]
    
    return similar_product_names

# Example usage:
# Define the product ID for which you want to find similar products
query_product_id = 980  # Replace 0 with the actual ID of the product

# Example usage:
# Find similar products
similar_products = find_similar_products(similarity_matrix_sparse, perfumes, query_product_id)
print("Products similar to", perfumes.loc[query_product_id, 'name'], "are:", similar_products)




Products similar to MAGENTA TANZANITE are: ['REVE DE COCO Adopt', 'SAVAGE JASMINE Sana Jardin', 'ALYSSA ASHLEY AMBRE GRIS Alyssa Ashley', 'SISTERLAND BLUE NEROLI Benetton', 'KEEMUN Welton London']


In [61]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the perfumes DataFrame
perfumes = pd.read_csv(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\perfumes_batch1.csv")  # Assuming 'perfumes.csv' is the filename

# Fill missing values in 'Ingredients' column with an empty string
perfumes['Ingredients'].fillna('', inplace=True)

# Define a function to tokenize each ingredient based on commas
def tokenize_ingredients(ingredients):
    return ingredients.split(', ')

# Initialize TF-IDF Vectorizer with custom tokenizer
vectorizer = TfidfVectorizer(tokenizer=tokenize_ingredients, lowercase=False)

# Fit and transform the ingredients column
ingredient_matrix = vectorizer.fit_transform(perfumes['Ingredients'])

# Calculate cosine similarity between ingredient vectors
cos_sim_matrix = cosine_similarity(ingredient_matrix)

# Initialize DataFrame to store results
results = pd.DataFrame(columns=[
    'perfume_index', 'Absolute URLs_x', 'Name_x', 'Brand_x', 
    'Ingredients_x', 'Classification_x', 'Family_and_Subfamily_x', 
    'Description_x', 'similar_perfume_index', 'rounded_percentage_match',
    'absolute_urls', 'name', 'brand', 'ingredients', 
    'classification', 'family_and_subfamily', 'description'
])

# Iterate through the similarity matrix
for i in range(len(perfumes)):
    similarities = list(enumerate(cos_sim_matrix[i]))
    similarities.sort(key=lambda x: x[1], reverse=True)  # Sort by similarity score
    
    # Exclude self-similarity (matching with the same perfume)
    top_3_similarities = [similarity for similarity in similarities[1:4] if similarity[0] != i]
    
    for j, similarity in top_3_similarities:
        rounded_similarity = round(similarity * 100)
        perfume1_info = perfumes.iloc[i]
        perfume2_info = perfumes.iloc[j]
        results = results.append({
            'perfume_index': i,
            'Absolute URLs_x': perfume1_info['Absolute URLs'],
            'Name_x': perfume1_info['Name'],
            'Brand_x': perfume1_info['Brand'],
            'Ingredients_x': perfume1_info['Ingredients'],
            'Classification_x': perfume1_info['Classification'],
            'Family_and_Subfamily_x': perfume1_info['Family_and_Subfamily'],
            'Description_x': perfume1_info['Description'],
            'similar_perfume_index': j,
            'rounded_percentage_match': rounded_similarity,
            'absolute_urls': perfume2_info['Absolute URLs'],
            'name': perfume2_info['Name'],
            'brand': perfume2_info['Brand'],
            'ingredients': perfume2_info['Ingredients'],
            'classification': perfume2_info['Classification'],
            'family_and_subfamily': perfume2_info['Family_and_Subfamily'],
            'description': perfume2_info['Description']
        }, ignore_index=True)

# Save results to CSV
results.to_csv('perfume_similarity_results_top3_nlp.csv', index=False)





In [48]:
result = similar_perfumes_df

In [49]:
result = pd.merge(result, perfumes, left_on='perfume_index', right_index=True, how='left')
result

,perfume_index,similar_perfume_index,rounded_percentage_match,absolute_urls,name,brand,ingredients,classification,family_and_subfamily,description
0,0,4926,51,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed..."
1,0,7104,45,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed..."
2,0,5207,45,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed..."
3,0,2033,43,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed..."
4,1,6093,77,https://www.wikiparfum.com/en/fragrances/o-fem...,O FEMININ,M. Micallef,"Rose, Neroli, Musk, Pink Pepper, Patchouli, Le...",Floral Crisp Citrus,"FLORAL, CITRUS","Sensual, carnal and modern, this chic and bohe..."
...,...,...,...,...,...,...,...,...,...,...
43271,10817,9661,52,https://www.wikiparfum.com/en/fragrances/per-se,PER SE,27 87,"Musk, Bamboo, Ambrette Seed, Pink Pepper, Viol...",Soft Floral Crisp Musk,"MUSK SKIN, FLORAL","Crafting a symphony of achromatic elegance, Pe..."
43272,10818,3174,66,https://www.wikiparfum.com/en/fragrances/mosaique,MOSAIQUE,27 87,"Plum, Honey, Oakmoss, Patchouli, Mate, Cocoa, ...",Mossy Woods Crisp Gourmand,"GOURMAND, CHYPRE",A cold smoky vibe infused with the mystical sw...
43273,10818,2955,64,https://www.wikiparfum.com/en/fragrances/mosaique,MOSAIQUE,27 87,"Plum, Honey, Oakmoss, Patchouli, Mate, Cocoa, ...",Mossy Woods Crisp Gourmand,"GOURMAND, CHYPRE",A cold smoky vibe infused with the mystical sw...
43274,10818,1457,59,https://www.wikiparfum.com/en/fragrances/mosaique,MOSAIQUE,27 87,"Plum, Honey, Oakmoss, Patchouli, Mate, Cocoa, ...",Mossy Woods Crisp Gourmand,"GOURMAND, CHYPRE",A cold smoky vibe infused with the mystical sw...


In [50]:
result.to_csv('result1testetstest.csv')

In [15]:
result2 = pd.read_csv(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\result1.csv")


In [16]:
result2 = pd.merge(result2, perfumes, left_on='similar_perfume_index', right_index=True, how='left')
result2

,index,perfume_index,Absolute URLs_x,Name_x,Brand_x,Ingredients_x,Classification_x,Family_and_Subfamily_x,Description_x,similar_perfume_index,rounded_percentage_match,absolute_urls,name,brand,ingredients,classification,family_and_subfamily,description
0,0,0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed...",4923,51,https://www.wikiparfum.com/en/fragrances/betel...,BETELGEUSE,In Astra,"Coffee, Oakmoss, Osmanthus, Iris / Orris, Cori...",Soft Floral Crisp Iris,"FLORAL, ALDEHYDIC",Betelgeuse seduces those who can savour its tr...
1,1,0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed...",7100,45,https://www.wikiparfum.com/en/fragrances/legend-1,LEGEND,Michael Jordan,"Musk, Coffee, Patchouli, Lavender, Bergamot, C...",Woods Classical,WOODY,Legend by Michael Jordan is a masculine fragra...
2,2,0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed...",5204,45,https://www.wikiparfum.com/en/fragrances/ajar,AJAR,Jardins d'Ecrivains,"Pear, Oakmoss, Bergamot, Orchid, Jasmine, Ince...",Woods Fresh Fruity,"FRUITY, WOODY",Romain Gary achieved literary subterfuge when ...
3,3,1,https://www.wikiparfum.com/en/fragrances/o-fem...,O FEMININ,M. Micallef,"Rose, Neroli, Musk, Pink Pepper, Patchouli, Le...",Floral Crisp Citrus,"FLORAL, CITRUS","Sensual, carnal and modern, this chic and bohe...",6090,77,https://www.wikiparfum.com/en/fragrances/midni...,MIDNIGHT RAIN,La Prairie,"Musk, Pomegranate, Patchouli, Lily, Guava, Vet...",Woody Amber Classical,"AMBERY (ORIENTAL), WOODY","The embodiment of free-spirited passion, Midni..."
4,4,1,https://www.wikiparfum.com/en/fragrances/o-fem...,O FEMININ,M. Micallef,"Rose, Neroli, Musk, Pink Pepper, Patchouli, Le...",Floral Crisp Citrus,"FLORAL, CITRUS","Sensual, carnal and modern, this chic and bohe...",5452,71,https://www.wikiparfum.com/en/fragrances/sun-m...,SUN MEN SPORT,Jil Sander,"Violet Leaves, Cardamom, Bergamot, Cyclamen, A...",Aromatic Fougere Fresh Spicy,"SPICY, AROMATIC FOUGERE",The sun is the limit with Jil Sander Sun Men S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32422,32422,10807,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR PLATINUM,24,"Rose, Chamomile, Juniper Berries, Lemon, Incen...",Dry Woods Classical,"WOODY, LEATHER","As I wandered through the surreal garden, my t...",224,54,https://www.wikiparfum.com/en/fragrances/ibiza...,IBIZA BOHEMIA,Adopt,"Coconut, Tuberose, Skin Accord, Jasmine",Floral Amber Crisp White Flower,"AMBERY (ORIENTAL), FLORAL",Bring light and the bohemian spirit into your ...
32423,32423,10807,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR PLATINUM,24,"Rose, Chamomile, Juniper Berries, Lemon, Incen...",Dry Woods Classical,"WOODY, LEATHER","As I wandered through the surreal garden, my t...",395,53,https://www.wikiparfum.com/en/fragrances/n-10-...,No 10 WHITE OUD,Agonist,"Musk, Violet, Rose (Grasse), Patchouli, Ambrar...",Soft Floral Classical Musk,"MUSK SKIN, FLORAL","To celebrate their 10th anniversary ""No 10 Whi..."
32424,32424,10808,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR GOLD,24,"Guaiac Wood, Honey, Saffron, Cardamom, Cedarwo...",Woods Classical,WOODY,"I rose with the sun, carried by an imperceptib...",3171,66,https://www.wikiparfum.com/en/fragrances/only-...,ONLY THE BRAVE,Diesel,"Tangerine, Violet Leaves, Rosemary, Leaves, Le...",Woody Amber Fresh,"AMBERY (ORIENTAL), WOODY",It is a question of conviction. A question of ...
32425,32425,10808,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR GOLD,24,"Gu

In [34]:
result2.to_csv('final_result.csv', index = False)
